In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Seed for reproducibility
np.random.seed(42)

# Generate random electric vehicle dataset
n_samples = 1000
features = {
    'Battery Capacity (kWh)': np.random.uniform(20, 100, n_samples),
    'Charging Time (hours)': np.random.uniform(0.5, 12, n_samples),
    'Range per Charge (miles)': np.random.uniform(100, 400, n_samples)
}

# Simulate binary classification labels (High vs. Low Performance)
labels = np.random.choice([0, 1], size=n_samples, p=[0.5, 0.5])

# Create DataFrame
data = pd.DataFrame(features)
data['Performance Label'] = labels

# Train-test split
X = data.drop('Performance Label', axis=1)
y = data['Performance Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Output data for reference
print("Sample training data:")
print(pd.DataFrame(X_train_scaled, columns=X.columns).head())

# Save dataset to a CSV file
data.to_csv('electric_vehicle_dataset.csv', index=False)

# Google Colab, download the file
from google.colab import files
files.download('electric_vehicle_dataset.csv')  # Use the same file name here

Sample training data:
   Battery Capacity (kWh)  Charging Time (hours)  Range per Charge (miles)
0               -1.534059               1.077829                  0.507474
1                1.536054              -1.345349                  1.129535
2                0.394066              -1.529440                  0.040357
3               -1.594441              -0.354381                 -0.718223
4                1.433799              -1.324067                  1.679497


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Function to build an MLP
def build_mlp(input_dim, depth, width):
    model = Sequential()
    model.add(Dense(width, activation='relu', input_dim=input_dim))
    for _ in range(depth - 1):
        model.add(Dense(width, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Experiment with varying depth and width
configurations = [
    {'depth': 2, 'width': 16},
    {'depth': 4, 'width': 16},
    {'depth': 2, 'width': 64},
    {'depth': 4, 'width': 64}
]

results = []

for config in configurations:
    print(f"Training MLP with depth={config['depth']} and width={config['width']}")
    model = build_mlp(X_train_scaled.shape[1], config['depth'], config['width'])
    history = model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

    # Evaluate on test data
    loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
    results.append({'Depth': config['depth'], 'Width': config['width'], 'Test Accuracy': accuracy})

# Display results
results_df = pd.DataFrame(results)
print(results_df)

Training MLP with depth=2 and width=16


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training MLP with depth=4 and width=16
Training MLP with depth=2 and width=64
Training MLP with depth=4 and width=64
   Depth  Width  Test Accuracy
0      2     16          0.540
1      4     16          0.495
2      2     64          0.565
3      4     64          0.545
